In [2]:
import numpy as np
from numpy import loadtxt
import time
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
path = "/home/sam/projects/machine-learning/data/collaborative_filtering/small_movie_list.csv"
with open(path) as file:
    data = pd.read_csv(file)
data.head(10)

,Unnamed: 0,mean rating,number of ratings,title
0,0,3.400000,5,"Yards, The (2000)"
1,1,3.250000,6,Next Friday (2000)
2,2,2.000000,4,Supernova (2000)
3,3,2.000000,4,Down to You (2000)
4,4,2.672414,29,Scream 3 (2000)
5,5,4.220930,43,"Boondock Saints, The (2000)"
6,6,1.000000,1,Gun Shy (2000)
7,7,3.062500,32,"Beach, The (2000)"
8,8,2.300000,5,Snow Day (2000)
9,9,3.166667,3,"Tigger Movie, The (2000)"


In [36]:
def load_precalc_params_small():

    file = open("/home/sam/projects/machine-learning/data/collaborative_filtering/small_movies_X.csv", 'rb')
    X = loadtxt(file, delimiter = ",")

    file = open("/home/sam/projects/machine-learning/data/collaborative_filtering/small_movies_W.csv", 'rb')
    W = loadtxt(file,delimiter = ",")

    file = open("/home/sam/projects/machine-learning/data/collaborative_filtering/small_movies_b.csv", 'rb')
    b = loadtxt(file,delimiter = ",")
    b = b.reshape(1,-1)
    num_movies, num_features = X.shape
    num_users,_ = W.shape
    return(X, W, b, num_movies, num_features, num_users)
    
def load_ratings_small():
    file = open("/home/sam/projects/machine-learning/data/collaborative_filtering/small_movies_Y.csv", 'rb')
    Y = loadtxt(file,delimiter = ",")

    file = open("/home/sam/projects/machine-learning/data/collaborative_filtering/small_movies_R.csv", 'rb')
    R = loadtxt(file,delimiter = ",")
    return(Y,R)

def load_Movie_List_pd():
    """ returns df with and index of movies in the order they are in in the Y matrix """
    df = pd.read_csv('/home/sam/projects/machine-learning/data/collaborative_filtering/small_movie_list.csv', header=0, index_col=0,  delimiter=',', quotechar='"')
    mlist = df["title"].to_list()
    return(mlist, df)

X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


<h4>Cost Function for Collaborative Filtering Algorithm (loop version)</h4>

In [25]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    tick = time.time()
    nm, nu = Y.shape
    J = 0

    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j] # the matrix R i a binary indicator (filled with 0 and 1), where R(i,j)=1,
            # if user j gave a rating to movie i and R(i,j)=0 otherwise.
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    tock = time.time()
    return J, tock-tick

cost, t = cofi_cost_func(X, W, b, Y, R, 2)
print(cost)
print(t)

318399.41526138317
5.375514507293701


<h4>Cost Function for Collaborative Filtering Algorithm (vectorized version using numpy)</h4>

In [18]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    
    tick = time.time()
    nm, nu = Y.shape # mn = number of movies, nu = number of users

    predictions = np.matmul(X, W.T) + b
    error = np.multiply(predictions - Y, R)
    squared_error = np.sum(np.square(error))
    regularization = (lambda_ / 2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    J = (squared_error / 2) + regularization
    tock = time.time()
    
    return J, tock - tick

cost, t = cofi_cost_func(X, W, b, Y, R, 2)
print(cost)
print(t)


318399.41526138433
0.03490900993347168


<h4>Cost Function for Collaborative Filtering Algorithm (vectorized version using tensorflow)</h4>

In [19]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    tick = time.time()
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    tock = time.time()
    
    return J, tock-tick
cost, t = cofi_cost_func(X, W, b, Y, R, 2)
print(cost)
print(t)

318399.41526138433
0.0378108024597168


In [37]:
def compute_cost(X, W, b, Y, R, lambda_):

    Y_hat = tf.linalg.matmul(X, tf.transpose(W)) + b
    error = tf.math.multiply(Y_hat - Y, R)
    squared_error = tf.math.reduce_sum(tf.math.square(error))
    regularization = (lambda_ / 2) * (tf.math.reduce_sum(tf.math.square(W)) + tf.math.reduce_sum(tf.math.square(X)))

    cost = (squared_error / 2) + regularization
        
    return float(cost)

cost = compute_cost(X, W, b, Y, R, 2)
print(cost)   

318399.4152613844


In [20]:
def normalizeRatings(Y, R):
   
    Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R) 
    
    return(Ynorm, Ymean)

<h4>Implement Collaborative filtering algorithm with TensorFlow</h4>

In [21]:
def recommender_system(Y, R, num_features, epochs, alpha, lambda_):
    
    Ynorm, Ymean = normalizeRatings(Y, R)
    num_movies, num_users = Y.shape
    
    tf.random.set_seed(1234)
    W = tf.Variable(tf.random.normal((num_users, num_features),dtype=tf.float64), name='W')
    X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64), name='X')
    b = tf.Variable(tf.random.normal((1, num_users), dtype=tf.float64), name='b')
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=alpha)
    
    for i in range(epochs):
        
        with tf.GradientTape() as tape:

            cost_value, t = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

        grads = tape.gradient(cost_value, [X,W,b])
        optimizer.apply_gradients(zip(grads, [X,W,b]))  

    return W, X, b    

In [40]:
Y, R = load_ratings_small()
W, X, b = recommender_system(Y, R, 100, 300, 1e-1, 2)

In [24]:

def predict(X, W, b, Y, R):
    
    p = np.matmul(X, np.transpose(W)) + b
    Ynorm, Ymean = normalizeRatings(Y, R)
    pm = p + Ymean
    my_predictions = pm[:,0]
    ix = tf.argsort(my_predictions, direction='DESCENDING')
    return ix

ix = predict(X, W, b, Y, R)
print(ix)

tf.Tensor([1420 1393 3532 ... 4379 3005 4077], shape=(4778,), dtype=int32)
